In [1]:
#-----------------------------------------------------------------------------------------------------
# 문서들을 embed하는 예제
# - 여기서는 문서들을 전처리하고, 임베딩 하는 과정임.
#
# 질의 응답 시스템 과정
# 문서들 전처리 : 
#    단락별루 분할(\n\n) - 불용어 제거 -문장별루 분할.
# 임베딩 : 
#    kpf-sbert-v1.1로  문장 평균 임베딩벡터 구함 - es에 문장별루 단락text와 평균벡터 저장.
#-----------------------------------------------------------------------------------------------------

import os
import random
import numpy as np
import pandas as pd
import time
import random

from utils import MyUtils
from utils import get_sentences

myutils = MyUtils(yam_file_path='./data/settings_128.yaml')
myutils.seed_everything()  # seed 설정
DEVICE = myutils.GPU_info() # GPU 혹은 CPU

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

False
device: cpu


In [3]:
# DATA_FOLDER에 파일들을 불러와서 DF로 만듬.
# -문서는 \n으로 구분해야 하며, 맨앞에는 title이 와야 하고, \n 다음 문단들이 와야함. 문단들은 \n 구분됨.
# -예: 회사 개요\n{회사내용}\n{제품구성}

# 파일이 여러개인 경우 폴더 지정
#DATA_FOLDER = './doc/사규개정-out-renew/'
DATA_FOLDER = './doc/'

files = myutils.getListOfFiles(DATA_FOLDER)
assert len(files) > 0 # files가 0이면 assert 발생
print('*file_count: {}, file_list:{}'.format(len(files), files[0:5]))

# 파일 경로를 지정하면됨.
#files = ["../data11/mpower_doc/신입사원교육.txt",]
docid = 0  # **카운터가 문서에 uid가 되므로, 유일무이한 값므로 지정할것.
count = 0

titles = []
contextids = []
contexts = []

for idx, file_path in enumerate(files):
    if '.ipynb_checkpoints' not in file_path:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = f.read()
            #print(data)
            data_list = data.split('\n\n')  # '\n\n' 구분으로 다락 구분
            docid += 1
            count = 0
            # titles, contextids 임의로 구해서 데이터 프레임 만듬.
            for i in range(len(data_list)):
                if i > 0 and data_list[i]:
                    count += 1
                    titles.append(data_list[0].strip())  # 문서 제목은 맨처음 문장이고 \n\n로 구분되어야 함.
                    contextids.append(f'{docid}_{count}')
                    contexts.append(data_list[0].strip() + '\n' + data_list[i].strip()) # 두번째는 문서 제목+문서내용 합처서 문장만듬.
 
# 데이터 프레임으로 만듬.
df_contexts = pd.DataFrame((zip(contexts, titles, contextids)), columns = ['context','question', 'contextid'])     

print(f'*문단 총 계수:{len(df_contexts)}\n')

*file_count: 46, file_list:['./doc/직장 내 괴롭힘 예방 및 대응 규정.txt', './doc/회계업무 처리규정.txt', './doc/상벌 관리 지침.txt', './doc/감사위원회의 직무규정.txt', './doc/소프트웨어 관리 규정.txt']
*문단 총 계수:781



In [4]:
# JSON 파일로 저장해둠.
import json
examples = []

for context, title, contextid in zip(contexts, titles, contextids):
    doc = {}
    doc['context'] = context  # 문단
    doc['title'] = title      # 제목
    doc['contextid'] = contextid # 문서 id
                        
    examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("./data/moco_context.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

In [5]:
# 문장들로 분리
doc_sentences = get_sentences(df=df_contexts, remove_sentnece_len=1, remove_duplication=False)

print(f'len:{len(doc_sentences)}, 1: {doc_sentences[1]}')

  0%|                                                                                           | 0/781 [00:00<?, ?it/s][Kss]: Because there's no supported C++ morpheme analyzer, Kss will take pecab as a backend. :D
For your information, Kss also supports mecab backend.
We recommend you to install mecab or konlpy.tag.Mecab for faster execution of Kss.
Please refer to following web sites for details:
- mecab: https://github.com/hyunwoongko/python-mecab-kor
- konlpy.tag.Mecab: https://konlpy.org/en/latest/api/konlpy.tag/#mecab-class

100%|█████████████████████████████████████████████████████████████████████████████████| 781/781 [03:02<00:00,  4.29it/s]

*[get_sentences] 문장처리=>len:9, time:182.2287
*[get_sentences] 문장 길이=>평균:6.647887323943662 / MAX: 19 / MIN: 1

len:781, 1: ['직장 내 괴롭힘', '예방 및 대응 규정', ' 직장 내 괴롭힘', '행위의 금지', '① 직장 내 괴롭힘', '행위란 임, 직원이 직장에서의 지위 또는 관계 등의 우위를 이용하여 업무상 적정범위를 넘어 다른 직원에게 신체적, 정신적 고통을 주거나 근무환경을 악화시키는 행위를 말한다.', '② 누구든지 직장 내 괴롭힘', '행위를 하여서는 아니 된다.']


In [6]:
# 인덱스 추가
from tqdm.notebook import tqdm
from utils import embed_text, bi_encoder, mpower_index_batch, create_index, clustering_embedding, kmedoids_clustering_embedding

# ES 관련
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk
    
# 조건에 맞게 임베딩 처리하는 함수 
def embedding(paragraphs:list)->list:
    # 한 문단에 대한 40개 문장 배열들을 한꺼번에 임베딩 처리함
    embeddings = embed_text(model=BI_ENCODER1, paragraphs=paragraphs, return_tensor=False).astype(FLOAT_TYPE)    
    return embeddings

#---------------------------------------------------------------------------
#문단에 문장들의 임베딩을 구하여 각각 클러스터링 처리함.
#---------------------------------------------------------------------------
def index_data(es, df_contexts, doc_sentences:list):
    #클러스터링 계수는 문단의 계수보다는 커야 함. 
    #assert num_clusters <= len(doc_sentences), f"num_clusters:{num_clusters} > len(doc_sentences):{len(doc_sentences)}"
    #-------------------------------------------------------------
    # 각 문단의 문장들에 벡터를 구하고 리스트에 저장해 둠.
    start = time.time()
    cluster_list = []

    rfile_names = df_contexts['contextid'].values.tolist()
    rfile_texts = df_contexts['context'].values.tolist()

    if OUT_DIMENSION == 0:
        dimension = 768
    else:
        dimension = 128

    clustering_num = NUM_CLUSTERS
        
    docs = []
    count = 0
    for i, sentences in enumerate(tqdm(doc_sentences)):
        embeddings = embedding(sentences)
        if i < 3:
            print(f'[{i}] sentences-------------------')
            if len(sentences) > 5:
                print(sentences[:5])
            else:
                print(sentences)
                
            myutils.log_message(f'*[index_data] embeddings.shape: {embeddings.shape}', log_folder='./log/documet')
            print()
        
        #----------------------------------------------------------------
        multiple = 1
        
        # [bong][2023-04-28] 임베딩 출력 계수에 따라 클러스터링 계수를 달리함.
        if NUM_CLUSTERS_VARIABLE == True:
            embeddings_len = embeddings.shape[0]
            if embeddings_len > 2000:
                multiple = 6
            elif embeddings_len > 1000:
                multiple = 5 # 5배
            elif embeddings_len > 600:
                multiple = 4 # 4배
            elif embeddings_len > 300:
                multiple = 3 # 3배
            elif embeddings_len > 100:
                multiple = 2 # 2배
        #----------------------------------------------------------------
        
        # 0=문장클러스터링 임베딩
        if EMBEDDING_METHOD == 0:
            if CLUSTRING_MODE == "kmeans":
                # 각 문단에 분할한 문장들의 임베딩 값을 입력해서 클러스터링 하고 평균값을 구함.
                # [bong][2023-04-28] 문장이 많은 경우에는 클러스터링 계수를 2,3배수로 함
                emb = clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            else:
                emb = kmedoids_clustering_embedding(embeddings = embeddings, outmode=OUTMODE, num_clusters=(clustering_num*multiple), seed=SEED).astype(FLOAT_TYPE) 
            
        # 1= 문장평균임베딩
        elif EMBEDDING_METHOD == 1:
            # 문장들에 대해 임베딩 값을 구하고 평균 구함.
            arr = np.array(embeddings).astype(FLOAT_TYPE)
            emb = arr.mean(axis=0).reshape(1,-1) #(128,) 배열을 (1,128) 형태로 만들기 위해 reshape 해줌
            clustering_num = 1  # 평균값일때는 NUM_CLUSTERS=1로 해줌.
        # 2=문장임베딩
        else:
            emb = embeddings

        if i < 3:
            myutils.log_message(f'*[index_data] cluster emb.shape: {emb.shape}', log_folder='./log/documet')
            print()
        
        #--------------------------------------------------- 
        # docs에 저장 
        #  [bong][2023-04-28] 여러개 벡터인 경우에는 벡터를 10개씩 분리해서 여러개 docs를 만듬.
        for j in range(multiple):
            count += 1
            doc = {}                                #dict 선언
            doc['rfile_name'] = rfile_names[i]      # contextid 담음
            doc['rfile_text'] = rfile_texts[i]      # text 담음.
            doc['dense_vectors'] = emb[j * clustering_num : (j+1) * clustering_num] # emb 담음.
            docs.append(doc)
        #---------------------------------------------------    

            if count % BATCH_SIZE == 0:
                mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
                docs = []
                myutils.log_message("[index_data](1) Indexed {} documents.".format(count), log_folder='./log/documet')
                 

    if docs:
        mpower_index_batch(es, ES_INDEX_NAME, docs, vector_len=clustering_num, dim_size=dimension)
        myutils.log_message("[index_data](2) Indexed {} documents.".format(count), log_folder='./log/documet')

    es.indices.refresh(index=ES_INDEX_NAME)

    myutils.log_message(f'*인덱싱 시간 : {time.time()-start:.4f}\n', log_folder='./log/documet')
    print()
#---------------------------------------------------------------------------


In [7]:
# param--------------------------------------------------------------------
OUT_DIMENSION =128   # 128 혹은 768이면 0입력
EMBEDDING_METHOD=0  # 0=클러스터링 임베딩, 1=문장평균임베딩, 2=문장임베딩
NUM_CLUSTERS=10     # 클러스터링 임베딩일때 클러스터링 수 
NUM_CLUSTERS_VARIABLE=False # 클러스터링 임베딩일때 클러스터링을 문장계수마다 다르계할지.
CLUSTRING_MODE = "kmeans"  # "kmeans" = k-평균 군집 분석, kmedoids =  k-대표값 군집 분석
OUTMODE = "mean"           # kmeans 일때=>mean=평균임베딩값, max=최대임베딩값.,  kmedoids 일때 =>mean=평균임베딩값, medoid=대표임베딩값.

MODEL_PATH = './model/kpf-sbert-128d-v1'#'../../../data11/model/kpf-sbert-v1.1'
POLLING_MODE = 'mean' # 폴링모드 

FLOAT_TYPE = 'float16' # float32 혹은 float16
SEED = 111

# ES 접속
ES_URL = 'http://192.168.0.102:9200/'               # es 접속 주소
ES_INDEX_NAME = 'qaindex_128_10'          # 생성혹은 추가할 인덱스 명
ES_INDEX_FILE = './data/mpower10u_128d_10.json'   # 인덱스 구조 파일경로
BATCH_SIZE=20       # ES 배치 사이즈
CREATE_INDEX = True # True이면 기존에 인덱스가 있다면 제거하고 다시 생성.
# param--------------------------------------------------------------------

es = Elasticsearch(ES_URL)
create_index(es, ES_INDEX_FILE, ES_INDEX_NAME, create=CREATE_INDEX)

# 임베딩 모델 로딩
WORD_EMBDDING_MODEL1, BI_ENCODER1 = bi_encoder(model_path=MODEL_PATH, max_seq_len=512, do_lower_case=True, 
                                               pooling_mode=POLLING_MODE, out_dimension=OUT_DIMENSION, device=DEVICE)

print(BI_ENCODER1)
print()
try:
    index_data(es, df_contexts, doc_sentences)
except Exception as e:
    error = f'index_data fail'
    msg = f'{error}=>{e}'
    myutils.log_message(f'/embed/es {msg}', log_folder='./log/documet')

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


1:{			"settings": {
				"number_of_shards": 2,
				"number_of_replicas": 1,
				"index": {
					"merge": {
						"scheduler": {
							"max_thread_count": 1
						}
					}
				},
				"analysis": {
					"analyzer": {
						"mpower10u_korean_analyzer": {
							"type": "custom",
							"tokenizer": "mpower10u_korean_tokenizer",
							"filter": [
								"lowercase",
								"stop",
								"nori_readingform",
								"mpower10u_korean_pos"
							]
						}
					},
					"tokenizer": {
						"mpower10u_korean_tokenizer": {
							"type": "nori_tokenizer",
							"decompound_mode": "discard"
						}
					},
					"filter": {
						"mpower10u_korean_pos": {
							"type": "nori_part_of_speech",
							"stoptags": [
								"J", "E", "XSA", "XSN", "XSV"
							]
						}
					}
				}
			},
			"mappings": {
				"_source": {
					"enabled": true
				},
				"dynamic":"strict",
				"properties": {
					"rfile_name": {
						"type": "text",
						"store": true
					},
					"rfile_text": {
						"ty

  0%|          | 0/781 [00:00<?, ?it/s]

[0] sentences-------------------
['직장 내 괴롭힘', '예방 및 대응 규정', ' 목적', '회사는 직장 내에서 괴롭힘', '행위를 예방하여 직원들이 안전하게 근로할 수 있도록 이 규정을 시행한다.']
[2023-12-07 17:13:35]*[index_data] embeddings.shape: (9, 128)


[2023-12-07 17:13:35]*[index_data] cluster emb.shape: (9, 128)


[1] sentences-------------------
['직장 내 괴롭힘', '예방 및 대응 규정', ' 직장 내 괴롭힘', '행위의 금지', '① 직장 내 괴롭힘']
[2023-12-07 17:13:36]*[index_data] embeddings.shape: (8, 128)


[2023-12-07 17:13:36]*[index_data] cluster emb.shape: (8, 128)


[2] sentences-------------------
['직장 내 괴롭힘', '예방 및 대응 규정', ' 금지되는 직장 내 괴롭힘', '행위', '회사에서 금지되는 직장 내 괴롭힘']
[2023-12-07 17:13:37]*[index_data] embeddings.shape: (11, 128)




/usr/local/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

  0%|          | 0/20 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[2023-12-07 17:21:45][index_data](1) Indexed 780 documents.



  0%|          | 0/1 [00:00<?, ?it/s]

[2023-12-07 17:21:46][index_data](2) Indexed 781 documents.

[2023-12-07 17:21:47]*인덱싱 시간 : 493.1275



